# Selfie to Anime Project

 **Code 0. Import Library**

In [1]:
import glob
import random
import os

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch

import os
import numpy as np
import math
import itertools
import datetime
import time

from torchvision.utils import save_image, make_grid
from torchvision import datasets
from torch.autograd import Variable

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Code 1. Dataset**

In [3]:
# 이미지 RGB형식으로 변환
def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image

In [4]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned #unaligned=False이면 비정렬형태 (1:1로 매칭x)
        # train 모드일 때는 trainA, trainB에 있는 디렉토리에서 이미지를 불러옵니다.
        if mode=="train":
            # glob 함수로 trainA 디렉토리의 이미지의 목록을 불러옵니다.
            self.files_A = sorted(glob.glob(os.path.join(root, "trainA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "trainB") + "/*.*"))
        else:
            self.files_A = sorted(glob.glob(os.path.join(root, "testA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "testB") + "/*.*"))

    #학습할 이미지를 랜덤으로 불러오기
    def __getitem__(self, index):
        # index값으로 이미지의목록 중 이미지 하나를 불러옵니다.
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        # unaligned 변수로 학습할 Pair를 랜덤으로 고릅니다. (1:1 pair로 매칭되는 형태는 아니므로 아무거나 랜덤으로 고르는 게 가능함)
        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert grayscale images to rgb
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)

        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

**Code 2. Generator & Discriminator**

In [5]:
#layer별 가중치 초기화
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1: #Conv layer일 때
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1: #BatchNorm2d layer일 때
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), #가장 가까운 픽셀 값을 복사해서 padding에 이용함 -> zero padding보다 훨씬 더 자연스러운 이미지를 생성함
            nn.Conv2d(in_features, in_features, 3), ##nn.Conv2d()의 인자 -> 첫번째 인자 = input channel수 = filter depth, 두번째 인자 = output channel수 = filter 개수, 세번째 인자 = filter크기 (3x3)
            nn.InstanceNorm2d(in_features), #개별 이미지를 정규화
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )
    # 중요 부분 (이전 layer의 출력값을 단순히 다음 layer에 더해서, gradient vanishing 문제를 해결한 부분) (Residual connection)
    def forward(self, x):
        return x + self.block(x)

In [7]:
# ResNet을 적용한 GAN #중요 부분
# 이 과정을 거치면 이미지 feature가 추출됨
class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()

        channels = input_shape[0]

        # Initial convolution block
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # Downsampling -> input image의 특징을 압축 추출하는 부분!! #중요
        # 정보를 응축시키고 residual block을 이용해서 최대한 많은 특징을 추출해내는 부분
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), #stride = 2인 DownSampling을 2번하므로 이미지 크기가 1/4배가 됨
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]

        # Upsampling -> 이미지의 스타일을 변경(강조)해주는 용도 #중요
        # 응축해놨던 정보를 펼치는 부분!
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2), #이미지 2배 확대
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer -> 최종적으로 추출된 특징으로부터 이미지를 생성하는 부분
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [8]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)] #Down sampling을 통해 출력 image의 크기 줄이기
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        ) #discriminator block 4번 통과해서 이미지 사이즈가 1/16으로 줄어듦 (왜냐면 stride가 2), 그리고 채널수는 discriminator block을 통과할수록 증가

    def forward(self, img):
        return self.model(img)

**Code 3. Training**

In [9]:
dataset_name="selfie2anime"
channels = 3
img_height = 256
img_width = 256
n_residual_blocks=9
lr=0.0002
b1=0.5
b2=0.999
n_epochs=20
init_epoch=0
decay_epoch=100
lambda_cyc=10.0
lambda_id=5.0
n_cpu=8
batch_size=4
sample_interval=100
checkpoint_interval=5

In [10]:
# Create sample and checkpoint directories
os.makedirs("images/%s" % dataset_name, exist_ok=True)
os.makedirs("saved_models/%s" % dataset_name, exist_ok=True)

In [11]:
!pwd

/content


In [ ]:
#!unzip -qq "../images/selfie2anime/archive.zip" -d "../images/selfie2anime/"

unzip:  cannot find or open ../images/selfie2anime/archive.zip, ../images/selfie2anime/archive.zip.zip or ../images/selfie2anime/archive.zip.ZIP.


In [12]:
# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss() #!
criterion_identity = torch.nn.L1Loss()

In [13]:
input_shape = (channels, img_height, img_width)

# Initialize generator and discriminator (모델 선언)
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)

In [14]:
cuda = torch.cuda.is_available()

if cuda:
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

In [15]:
# Initialize weights
G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d((1, 0, 1, 0))
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [16]:
# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))

In [17]:
# learning rate scheduler
class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        #assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

In [18]:
# Learning rate update schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)

In [19]:
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [20]:
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [21]:
# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

In [22]:
# Image transformations
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

In [23]:
# Training data loader
dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/archive/" , transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)
# Test data loader
val_dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/archive/", transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs["B"].type(Tensor))
    fake_A = G_BA(real_B)
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    save_image(image_grid, "images/%s/%s.png" % (dataset_name, batches_done), normalize=False)

In [25]:
# 이 부분이 제일 중요함!!
prev_time = time.time()
for epoch in range(1, n_epochs+1):
    for i, batch in enumerate(dataloader):

        # (1) Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))

        # (2) Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False) #진짜를 나타내는 label
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False) #가짜를 나타내는 label

        # (3) Train Generators

        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # (4) Identity loss (L1 loss)
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # Generator는 MSE loss -> L1 loss를 이용
        # (5) GAN loss (MSE loss) #중요
        fake_B = G_AB(real_A) # B hat
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid) #생성된 이미지를 판별한 결과와 실제 이미지 label의 차이를 계산해서,
                                                        #생성된 이미지가 진짜와 유사하게 보이도록 'Generator'를 훈련시키는 데 사용됨
                                                        #D_B가 가짜라고 판단하면, Discriminator는 잘 판단한 거지만 Generator가 잘 생성하지 못한 거여서 이 loss값이 클 것이고,
                                                        #Generator를 더 훈련시키는 데에 이용됨
        fake_A = G_BA(real_B) # A hat
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # (6) Cycle loss (L1 loss) #중요
        recov_A = G_BA(fake_B) # a hat
        loss_cycle_A = criterion_cycle(recov_A, real_A) #원본 이미지와 복원 이미지의 차이값
        recov_B = G_AB(fake_A) # b hat
        loss_cycle_B = criterion_cycle(recov_B, real_B) #원본 이미지와 복원 이미지의 차이값

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # (7) Total loss
        loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity

        loss_G.backward()
        optimizer_G.step()
        #-----------------------------------------------------------------------#
        # (8) Train Discriminator A

        optimizer_D_A.zero_grad()

        # (9) Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # (10) Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # (11) Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # (12) Train Discriminator B

        optimizer_D_B.zero_grad()

        # (13) Real loss
        loss_real = criterion_GAN(D_B(real_B), valid) #Discriminator의 능력을 평가 (loss값이 작으면 discriminator가 잘 판별하는 것)(평가에 이용되는 데이터가 실제이미지)

        # (14) Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake) #Discriminator의 능력을 평가 (loss값이 작으면 discriminator가 잘 판별하는 것)(평가에 이용되는 데이터가 가짜이미지)

        # (15) Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # (16) Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # (17) Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )

        # (18) If at sample interval save image
        if batches_done % sample_interval == 0:
            sample_images(batches_done)

    # (19) Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # (20) Save model checkpoints
    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (dataset_name, epoch))
        torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (dataset_name, epoch))
        torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (dataset_name, epoch))
        torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (dataset_name, epoch))

<ipython-input-25-e0659687f55f>:11: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False) #진짜를 나타내는 label


[Epoch 20/20] [Batch 852/853] [D loss: 0.088454] [G loss: 3.321859, adv: 0.847585, cycle: 0.176013, identity: 0.142830] ETA: -1 day, 23:54:16.816157